In [42]:
!mkdir -p backend/agents


In [43]:
%%writefile backend/requirements.txt
fastapi
uvicorn
google-genai
python-dotenv
nest_asyncio


Overwriting backend/requirements.txt


In Notebook A (where agents/*.py exist)

Zip the agents folder:

!zip -r agents_code.zip agents


In the Kaggle UI, go to the “Output” section on the right → you’ll see agents_code.zip.
Click it and Download.

🔹 In Notebook B (the backend notebook)

Upload agents_code.zip:

Right panel → Add data → Upload → drag agents_code.zip.

Kaggle will mount it under something like:

/kaggle/input/agentscodezip/agents_code.zip


Unzip into the working directory:

!unzip -o /kaggle/input/agentscodezip/agents_code.zip -d .


Now you’ll have ./agents/ in this notebook too.

In [44]:
!unzip -o /kaggle/input/agents.zip/agents.zip -d .

unzip:  cannot find or open /kaggle/input/agents.zip/agents.zip, /kaggle/input/agents.zip/agents.zip.zip or /kaggle/input/agents.zip/agents.zip.ZIP.


In [45]:
!mkdir -p backend/agents
!cp /kaggle/input/agents-zip/agents/*.py backend/agents/


In [46]:
import nest_asyncio
nest_asyncio.apply()


In [47]:
!pip install google-genai==0.2.0 fastapi uvicorn nest_asyncio
!pip install "google-cloud-aiplatform>=1.60.0"


  Using cached google_genai-1.52.0-py3-none-any.whl.metadata (46 kB)
Using cached google_genai-1.52.0-py3-none-any.whl (261 kB)
  Attempting uninstall: google-genai
    Found existing installation: google-genai 0.2.0
    Uninstalling google-genai-0.2.0:
      Successfully uninstalled google-genai-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.18.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 14.2 which is incompatible.


In [48]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR KEY"


In [49]:
%%writefile backend/main.py
import sys, os
sys.path.append(os.path.join(os.path.dirname(__file__), ""))
sys.path.append(os.path.join(os.path.dirname(__file__), "agents"))

import json
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional

from agents.orchestrator_agent import OrchestratorAgent
from agents.quiz_agent import QuizAgent
from agents.help_agent import HelpAgent

# ---------------------------
# Ensure data directory exists
# ---------------------------
DATA_DIR = "backend/data"
os.makedirs(DATA_DIR, exist_ok=True)

TASKS_FILE = os.path.join(DATA_DIR, "tasks.json")
PROFILE_FILE = os.path.join(DATA_DIR, "profile.json")

# ---------------------------
# Load / Save Helpers
# ---------------------------
def load_json(path, fallback):
    if not os.path.exists(path):
        return fallback
    try:
        with open(path, "r") as f:
            return json.load(f)
    except:
        return fallback

def save_json(path, data):
    with open(path, "w") as f:
        json.dump(data, f, indent=2)


# ---------------------------
# FastAPI App
# ---------------------------
app = FastAPI()

# Allow frontend access
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)


# ---------------------------
# Pydantic Models
# ---------------------------
class Task(BaseModel):
    id: str
    title: str
    description: str
    category: str
    status: str
    dueDate: str
    dueTime: str
    verificationFailedCount: int = 0
    aiAccess: Optional[str] = None

class UserProfile(BaseModel):
    name: str
    email: str
    bio: Optional[str] = ""
    character: str
    documentName: Optional[str] = None
    documentContent: Optional[str] = None


# ---------------------------
# API Request Models
# ---------------------------
class ProcessRequest(BaseModel):
    title: str
    description: str
    category: str
    contextUrl: Optional[str] = None
    userProfile: Optional[dict] = None

class SubmitQuizRequest(BaseModel):
    normalized: dict
    quiz: dict
    answers: List[int]

class VerifyNetworkingRequest(BaseModel):
    names: List[str]
    task: dict

class SpreadsheetRequest(BaseModel):
    tasks: List[dict]
    user: dict


# ---------------------------
# Agents (require API key)
# ---------------------------
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise RuntimeError("GOOGLE_API_KEY is missing")

orchestrator = OrchestratorAgent()
quiz_agent = QuizAgent(api_key=API_KEY)
help_agent = HelpAgent(api_key=API_KEY)


# ---------------------------
# DATA ENDPOINTS
# ---------------------------
@app.get("/tasks")
def get_tasks():
    return load_json(TASKS_FILE, [])

@app.post("/save-tasks")
def save_tasks(tasks: List[Task]):
    save_json(TASKS_FILE, [t.dict() for t in tasks])
    return {"status": "ok"}

@app.get("/profile")
def get_profile():
    return load_json(
        PROFILE_FILE,
        {
            "name": "",
            "email": "",
            "bio": "",
            "character": "MALE",
            "documentName": None,
            "documentContent": None,
        },
    )

@app.post("/save-profile")
def save_profile(profile: UserProfile):
    save_json(PROFILE_FILE, profile.dict())
    return {"status": "ok"}


# -----------------------------------------------------
# MAIN ROUTE USED BY FRONTEND FOR QUIZ + HELP
# -----------------------------------------------------
@app.post("/process-task")
async def process_task(payload: ProcessRequest):

    task = {
        "title": payload.title,
        "description": payload.description,
        "category": payload.category,
        "contextUrl": payload.contextUrl,
        "userProfile": payload.userProfile,
    }

    result = await orchestrator.process(task)
    return result


# -----------------------------------------------------
# QUIZ SUBMISSION → MEMORY OR HELP
# -----------------------------------------------------
@app.post("/submit-quiz")
async def submit_quiz(payload: SubmitQuizRequest):
    result = await orchestrator.submit_quiz(
        payload.normalized, payload.quiz, payload.answers
    )
    return result


# -----------------------------------------------------
# NETWORKING VERIFICATION
# -----------------------------------------------------
@app.post("/verify-networking")
async def verify_networking(payload: VerifyNetworkingRequest):
    names = payload.names
    task = payload.task

    # HelpAgent provides name verification capability
    try:
        result = await help_agent.verify_names(names, task)
        return {"verified": result}
    except:
        return {"verified": True}  # fallback


# -----------------------------------------------------
# SPREADSHEET GENERATION
# -----------------------------------------------------
@app.post("/generate-spreadsheet")
async def generate_spreadsheet(payload: SpreadsheetRequest):

    tasks = payload.tasks
    user = payload.user

    # We reuse HelpAgent to generate CSV using Gemini
    try:
        result = await help_agent.generate_spreadsheet(tasks, user)
        return result
    except Exception as e:
        return {"title": "Error.csv", "content": f"Error: {e}"}


# ---------------------------
# Root Test Endpoint
# ---------------------------
@app.get("/")
def root():
    return {"status": "backend running"}


Overwriting backend/main.py


In [50]:
!ls backend
!ls backend/agents


agents	data  main.py  __pycache__  requirements.txt
help_agent.py	 orchestrator_agent.py	quiz_agent.py
memory_agent.py  __pycache__		task_agent.py


In [51]:
!mkdir -p frontend/src/views
!mkdir -p frontend/src/services
!mkdir -p frontend/src/components
!mkdir -p frontend/src/types


In [52]:
%%writefile frontend/src/services/geminiService.ts
// FRONTEND SERVICE LAYER – Calls FastAPI backend instead of Gemini directly.

import { Task, UserProfile } from "../types";

// The backend URL (Kaggle notebook runs FastAPI at localhost:8000)
const API_BASE = "http://127.0.0.1:8000";

// Helper: POST wrapper
async function postJson<T>(path: string, body: any): Promise<T> {
  const res = await fetch(`${API_BASE}${path}`, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(body)
  });

  if (!res.ok) {
    throw new Error(`Request failed: ${res.status}`);
  }

  return await res.json();
}

/* -------------------------------------------------------------------------- */
/*                               QUIZ GENERATION                               */
/* -------------------------------------------------------------------------- */

export async function generateQuizForTask(task: Task, contextUrl?: string) {
  return postJson("/process-task", {
    title: task.title,
    description: task.description,
    category: task.category,
    contextUrl
  }).then((resp) => resp.quiz?.questions || []);
}

/* -------------------------------------------------------------------------- */
/*                                  HELP FLOW                                 */
/* -------------------------------------------------------------------------- */

export async function getHelpForTask(task: Task, user: UserProfile) {
  return postJson("/process-task", {
    title: task.title,
    description: task.description,
    category: task.category,
    userProfile: user
  }).then((resp) => resp.help);
}

/* -------------------------------------------------------------------------- */
/*                           VERIFY NETWORKING NAMES                           */
/* -------------------------------------------------------------------------- */

export async function verifyNetworkingNames(names: string[], task: Task) {
  return postJson("/verify-networking", {
    names,
    task
  }).then((resp) => resp.verified);
}

/* -------------------------------------------------------------------------- */
/*                          GRADE QUIZ + MEMORY AGENT                          */
/* -------------------------------------------------------------------------- */

export async function submitQuizAnswers(
  normalizedTask: any,
  quiz: any,
  userAnswers: number[]
) {
  return postJson("/submit-quiz", {
    normalized: normalizedTask,
    quiz,
    answers: userAnswers
  });
}

/* -------------------------------------------------------------------------- */
/*                         SPREADSHEET / CSV GENERATION                        */
/* -------------------------------------------------------------------------- */

export async function generateSpreadsheet(tasks: Task[], user: UserProfile) {
  return postJson("/generate-spreadsheet", {
    tasks,
    user
  });
}


Overwriting frontend/src/services/geminiService.ts


In [53]:
%%writefile frontend/src/views/Home.tsx
import React, { useState } from 'react';
import { DragDropContext, Droppable, Draggable, DropResult } from '@hello-pangea/dnd';
import { Plus, GripVertical, Clock, Calendar as CalendarIcon, Edit, ArrowRight, Play } from 'lucide-react';
import { Task, TaskStatus, UserProfile } from '../types';
import { TaskModal } from '../components/CreateTaskModal';
import { CATEGORY_COLORS } from '../constants';
import { CharacterIllustration } from '../components/Illustrations';

interface HomeProps {
  tasks: Task[];
  setTasks: React.Dispatch<React.SetStateAction<Task[]>>;
  userProfile: UserProfile;
  setActiveTab: (tab: string) => void;
}

const columnConfig = {
  todo: { title: 'To Do', status: TaskStatus.TODO, color: 'bg-gray-500' },
  inProgress: { title: 'In Progress', status: TaskStatus.IN_PROGRESS, color: 'bg-primary' },
};

type ColumnId = keyof typeof columnConfig;

export const Home: React.FC<HomeProps> = ({ tasks, setTasks, userProfile, setActiveTab }) => {
  const [isModalOpen, setIsModalOpen] = useState(false);
  const [editingTask, setEditingTask] = useState<Task | null>(null);

  const handleDragEnd = (result: DropResult) => {
    const { source, destination } = result;

    if (!destination) return;

    if (source.droppableId === destination.droppableId && source.index === destination.index) {
      return;
    }

    const startColumnId = source.droppableId as ColumnId;
    const finishColumnId = destination.droppableId as ColumnId;

    const startStatus = columnConfig[startColumnId].status;
    const finishStatus = columnConfig[finishColumnId].status;

    if (startColumnId === finishColumnId) {
      const columnTasks = tasks.filter(t => t.status === startStatus);
      const otherTasks = tasks.filter(t => t.status !== startStatus);

      const [removed] = columnTasks.splice(source.index, 1);
      columnTasks.splice(destination.index, 0, removed);

      setTasks([...columnTasks, ...otherTasks]);
    } else {
      const sourceTasks = tasks.filter(t => t.status === startStatus);
      const destTasks = tasks.filter(t => t.status === finishStatus);
      const otherTasks = tasks.filter(t => t.status !== startStatus && t.status !== finishStatus);

      const [removed] = sourceTasks.splice(source.index, 1);
      removed.status = finishStatus;
      destTasks.splice(destination.index, 0, removed);

      setTasks([...sourceTasks, ...destTasks, ...otherTasks]);
    }
  };

  const handleSaveTask = (taskData: any) => {
    if (taskData.id) {
      setTasks(tasks.map(t => t.id === taskData.id ? { ...t, ...taskData } : t));
    } else {
      const newTask: Task = {
        ...taskData,
        id: `task-${Date.now()}`,
        order: tasks.length,
        verificationFailedCount: 0,
      };
      setTasks([...tasks, newTask]);
    }
  };

  const handleStartTask = (taskId: string) => {
    setTasks(tasks.map(t => t.id === taskId ? { ...t, status: TaskStatus.IN_PROGRESS } : t));
  };

  const openCreateModal = () => {
    setEditingTask(null);
    setIsModalOpen(true);
  };

  const openEditModal = (task: Task) => {
    setEditingTask(task);
    setIsModalOpen(true);
  };

  const columns = {
    todo: tasks.filter(t => t.status === TaskStatus.TODO),
    inProgress: tasks.filter(t => t.status === TaskStatus.IN_PROGRESS),
  };

  return (
    <div className="p-8 max-w-6xl mx-auto">
      <header className="flex justify-between items-center mb-10">
        <div>
          <h1 className="text-3xl font-bold text-white mb-2">My Tasks</h1>
          <p className="text-gray-400">Drag tasks to change their status. Keep momentum.</p>
        </div>
        <button
          onClick={openCreateModal}
          className="flex items-center gap-2 bg-secondary hover:bg-pink-600 text-white px-6 py-3 rounded-xl font-bold shadow-lg shadow-secondary/25 transition-all transform hover:scale-105"
        >
          <Plus className="w-5 h-5" />
          New Task
        </button>
      </header>

      <DragDropContext onDragEnd={handleDragEnd}>
        <div className="grid grid-cols-1 lg:grid-cols-2 gap-8 items-start">
          {(Object.keys(columns) as ColumnId[]).map(columnId => {
            const column = columnConfig[columnId];
            const columnTasks = columns[columnId];

            return (
              <div key={columnId} className="bg-card/50 rounded-2xl border border-gray-800 p-6 min-h-[500px] flex flex-col">
                <div className="flex items-center justify-between mb-6">
                  <h2 className="text-xl font-semibold text-gray-200 flex items-center gap-3">
                    <div className={`w-3 h-3 rounded-full ${column.color}`}></div>
                    {column.title}
                  </h2>
                  <span className="bg-gray-800 text-gray-400 px-3 py-1 rounded-full text-xs font-bold">
                    {columnTasks.length}
                  </span>
                </div>

                <Droppable droppableId={columnId}>
                  {(provided, snapshot) => (
                    <div
                      {...provided.droppableProps}
                      ref={provided.innerRef}
                      className={`space-y-4 flex-1 rounded-lg p-2 transition-colors ${
                        snapshot.isDraggingOver ? 'bg-primary/10' : 'bg-transparent'
                      }`}
                    >
                      {columnTasks.length === 0 && (
                        <div className="text-center py-12 text-gray-500 flex flex-col items-center h-full justify-center">
                          <CharacterIllustration character={userProfile.character} mood="Idle" className="w-40 h-40 mb-4" />
                          <p className="font-medium">
                            {columnId === 'todo' ? "All clear!" : "No active tasks."}
                          </p>
                          <p className="text-sm">
                            {columnId === 'todo' ? "Create a new task to get started." : "Drag a task here to begin."}
                          </p>
                        </div>
                      )}

                      {columnTasks.map((task, index) => (
                        <Draggable key={task.id} draggableId={task.id} index={index}>
                          {(provided, snapshot) => (
                            <div
                              ref={provided.innerRef}
                              {...provided.draggableProps}
                              {...provided.dragHandleProps}
                              className={`bg-gray-800/80 p-5 rounded-xl border border-gray-700 hover:border-gray-500 transition-all group ${
                                snapshot.isDragging ? 'shadow-2xl ring-2 ring-primary rotate-2 z-50 bg-gray-800' : ''
                              }`}
                            >
                              <div className="flex justify-between items-start mb-3">
                                <div className={`px-3 py-1 rounded-full text-xs font-bold text-white ${CATEGORY_COLORS[task.category]}`}>
                                  {task.category}
                                </div>
                                <div className="flex items-center gap-2">
                                  <button
                                    onClick={() => openEditModal(task)}
                                    className="opacity-0 group-hover:opacity-100 text-gray-500 hover:text-white transition-opacity"
                                  >
                                    <Edit className="w-4 h-4" />
                                  </button>
                                  <GripVertical className="text-gray-600 group-hover:text-gray-400 cursor-grab" />
                                </div>
                              </div>

                              <h3 className="text-lg font-medium text-white mb-2">{task.title}</h3>
                              <p className="text-gray-400 text-sm line-clamp-2 mb-4">{task.description}</p>

                              <div className="flex items-center gap-4 text-xs text-gray-500 border-t border-gray-700 pt-3">
                                <div className="flex items-center gap-1.5">
                                  <CalendarIcon className="w-3.5 h-3.5" /> {task.dueDate}
                                </div>
                                <div className="flex items-center gap-1.5">
                                  <Clock className="w-3.5 h-3.5" /> {task.dueTime}
                                </div>
                              </div>

                              {task.status === TaskStatus.TODO && (
                                <button
                                  onClick={() => handleStartTask(task.id)}
                                  className="w-full mt-4 py-2.5 bg-green-600 hover:bg-green-500 text-white rounded-lg text-sm font-bold transition-colors flex items-center justify-center gap-2 shadow-lg shadow-green-900/20"
                                >
                                  Start Task <Play className="w-4 h-4 fill-current" />
                                </button>
                              )}

                              {task.status === TaskStatus.IN_PROGRESS && (
                                <button
                                  onClick={() => setActiveTab('progress')}
                                  className="w-full mt-4 py-2.5 bg-primary/80 hover:bg-primary text-white rounded-lg text-sm font-bold transition-colors flex items-center justify-center gap-2 shadow-lg shadow-primary/20"
                                >
                                  Check In <ArrowRight className="w-4 h-4" />
                                </button>
                              )}
                            </div>
                          )}
                        </Draggable>
                      ))}

                      {provided.placeholder}
                    </div>
                  )}
                </Droppable>
              </div>
            );
          })}
        </div>
      </DragDropContext>

      <TaskModal
        isOpen={isModalOpen}
        onClose={() => setIsModalOpen(false)}
        onSubmit={handleSaveTask}
        taskToEdit={editingTask}
      />
    </div>
  );
};


Overwriting frontend/src/views/Home.tsx


In [54]:
%%writefile frontend/src/views/Calendar.tsx
import React, { useState } from 'react';
import { Task, TaskStatus } from '../types';
import { CATEGORY_COLORS } from '../constants';
import { format, startOfMonth, endOfMonth, eachDayOfInterval, isSameDay, isSameMonth, addMonths, subMonths } from 'date-fns';
import { ChevronLeft, ChevronRight } from 'lucide-react';

interface CalendarProps {
  tasks: Task[];
  setActiveTab: (tab: string) => void;
}

export const Calendar: React.FC<CalendarProps> = ({ tasks, setActiveTab }) => {
  const [currentDate, setCurrentDate] = useState(new Date());
  
  const monthStart = startOfMonth(currentDate);
  const monthEnd = endOfMonth(currentDate);
  const days = eachDayOfInterval({ start: monthStart, end: monthEnd });

  const handlePrevMonth = () => setCurrentDate(subMonths(currentDate, 1));
  const handleNextMonth = () => setCurrentDate(addMonths(currentDate, 1));

  const handleTaskClick = (task: Task) => {
    if (task.status === TaskStatus.IN_PROGRESS || task.status === TaskStatus.FAILED_VERIFICATION) {
      setActiveTab('progress');
    } else {
      setActiveTab('home');
    }
  };

  return (
    <div className="p-8 h-screen flex flex-col">
      <header className="flex justify-between items-center mb-8">
        <h1 className="text-3xl font-bold text-white">Calendar</h1>
        <div className="flex items-center gap-4">
          <button 
            onClick={handlePrevMonth}
            className="p-2 hover:bg-gray-800 rounded-full transition-colors text-gray-400 hover:text-white"
          >
            <ChevronLeft className="w-6 h-6" />
          </button>
          <div className="text-xl font-medium text-primary w-40 text-center select-none">
            {format(currentDate, 'MMMM yyyy')}
          </div>
          <button 
            onClick={handleNextMonth}
            className="p-2 hover:bg-gray-800 rounded-full transition-colors text-gray-400 hover:text-white"
          >
            <ChevronRight className="w-6 h-6" />
          </button>
        </div>
      </header>

      <div className="flex-1 bg-card border border-gray-800 rounded-2xl overflow-hidden flex flex-col shadow-xl">
        
        {/* Days Header */}
        <div className="grid grid-cols-7 border-b border-gray-800 bg-gray-900/50">
          {['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'].map(day => (
            <div key={day} className="py-4 text-center text-sm font-bold text-gray-500 uppercase tracking-wider">
              {day}
            </div>
          ))}
        </div>

        {/* Calendar Grid */}
        <div className="flex-1 grid grid-cols-7 auto-rows-fr bg-gray-900/20">
          {days.map((day, idx) => {
            const dayTasks = tasks.filter(t => isSameDay(new Date(t.dueDate), day));
            const isToday = isSameDay(day, new Date());

            return (
              <div 
                key={idx} 
                className={`border-b border-r border-gray-800 p-2 min-h-[100px] relative hover:bg-gray-800/30 transition-colors group ${
                  !isSameMonth(day, currentDate) ? 'bg-gray-900/50 text-gray-700' : 'text-gray-300'
                }`}
              >
                <span className={`text-sm font-medium p-1 rounded-full w-8 h-8 flex items-center justify-center ${
                  isToday ? 'bg-primary text-white shadow-lg shadow-primary/50' : 'text-gray-400 group-hover:text-white'
                }`}>
                  {format(day, 'd')}
                </span>
                
                <div className="mt-2 space-y-1 overflow-y-auto max-h-24 custom-scrollbar">
                  {dayTasks.map(task => (
                    <div 
                      key={task.id}
                      onClick={(e) => {
                        e.stopPropagation();
                        handleTaskClick(task);
                      }}
                      className={`text-xs px-2 py-1.5 rounded-md truncate text-white ${CATEGORY_COLORS[task.category]} shadow-sm cursor-pointer hover:opacity-80 transition-opacity`}
                      title={task.title}
                    >
                      {task.title}
                    </div>
                  ))}
                </div>
              </div>
            );
          })}
        </div>

      </div>
    </div>
  );
};


Overwriting frontend/src/views/Calendar.tsx


In [55]:
%%writefile frontend/src/views/Documents.tsx
import React, { useState } from 'react';
import { FileSpreadsheet, Download, Plus, Loader2, FileText } from 'lucide-react';
import { GeneratedDocument, Task, UserProfile } from '../types';
import { CharacterIllustration } from '../components/Illustrations';

interface DocumentsProps {
  documents: GeneratedDocument[];
  setDocuments: React.Dispatch<React.SetStateAction<GeneratedDocument[]>>;
  tasks: Task[];
  userProfile: UserProfile;
}

export const Documents: React.FC<DocumentsProps> = ({ documents, setDocuments, tasks, userProfile }) => {
  const [isGenerating, setIsGenerating] = useState(false);

  const handleGenerateReport = async () => {
    setIsGenerating(true);
    try {
      const response = await fetch("http://127.0.0.1:8000/generate-report", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({
          tasks,
          user: userProfile
        })
      });

      if (!response.ok) throw new Error("Backend error");
      const result = await response.json();

      const newDoc: GeneratedDocument = {
        id: `doc-${Date.now()}`,
        title: result.title,
        type: 'csv',
        date: new Date().toLocaleDateString(),
        content: result.content,
      };

      setDocuments([newDoc, ...documents]);

    } catch (e) {
      console.error("Failed to generate report", e);
      alert("Sorry, I couldn't generate the report right now.");
    } finally {
      setIsGenerating(false);
    }
  };

  const handleDownload = (doc: GeneratedDocument) => {
    const blob = new Blob([doc.content], { type: 'text/csv;charset=utf-8;' });
    const link = document.createElement('a');
    const url = URL.createObjectURL(blob);
    link.setAttribute('href', url);
    link.setAttribute('download', doc.title);
    link.style.visibility = 'hidden';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
  };

  return (
    <div className="p-8 max-w-6xl mx-auto">
      <header className="flex justify-between items-center mb-10">
        <div>
          <h1 className="text-3xl font-bold text-white mb-2">Documents</h1>
          <p className="text-gray-400">Your generated reports and spreadsheets.</p>
        </div>
        <button
          onClick={handleGenerateReport}
          disabled={isGenerating}
          className="flex items-center gap-2 bg-green-600 hover:bg-green-500 text-white px-6 py-3 rounded-xl font-bold shadow-lg shadow-green-900/25 transition-all transform hover:scale-105 disabled:opacity-50 disabled:cursor-not-allowed"
        >
          {isGenerating ? <Loader2 className="w-5 h-5 animate-spin" /> : <Plus className="w-5 h-5" />}
          {isGenerating ? 'Analyzing Data...' : 'Create Report'}
        </button>
      </header>

      {documents.length === 0 ? (
        <div className="text-center py-20 flex flex-col items-center bg-card/50 rounded-2xl border border-gray-800">
          <CharacterIllustration character={userProfile.character} mood="Thinking" className="w-48 h-48 mb-6" />
          <h2 className="text-xl font-bold text-white mb-2">No documents yet</h2>
          <p className="text-gray-400 max-w-md mx-auto mb-8">
            I can organize your tasks into a neat spreadsheet for you. Just hit "Create Report" and I'll handle the rest.
          </p>
          <button
            onClick={handleGenerateReport}
            disabled={isGenerating}
            className="text-primary font-bold hover:underline"
          >
            Generate my first report
          </button>
        </div>
      ) : (
        <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
          {documents.map((doc) => (
            <div 
              key={doc.id} 
              className="bg-card border border-gray-800 rounded-2xl p-6 group hover:border-green-500/50 transition-all hover:shadow-xl hover:shadow-green-900/10 flex flex-col"
            >
              <div className="flex items-start justify-between mb-6">
                <div className="p-3 bg-green-900/20 rounded-xl text-green-500 group-hover:text-green-400 group-hover:scale-110 transition-all">
                    <FileSpreadsheet className="w-8 h-8" />
                </div>
                <span className="text-xs font-medium text-gray-500 bg-gray-800 px-2 py-1 rounded-md">{doc.type.toUpperCase()}</span>
              </div>
              
              <div className="mb-6 flex-1">
                <h3 className="text-lg font-bold text-white mb-1 line-clamp-1" title={doc.title}>{doc.title}</h3>
                <p className="text-sm text-gray-500 flex items-center gap-2">
                  Created: {doc.date}
                </p>
              </div>

              <button 
                onClick={() => handleDownload(doc)}
                className="w-full py-3 bg-gray-800 hover:bg-green-600 text-gray-300 hover:text-white rounded-xl font-medium transition-all flex items-center justify-center gap-2 group-hover:bg-green-600 group-hover:text-white"
              >
                <Download className="w-4 h-4" /> Download
              </button>
            </div>
          ))}
        </div>
      )}
    </div>
  );
};


Overwriting frontend/src/views/Documents.tsx


In [56]:
%%writefile frontend/src/views/Profile.tsx
import React, { useState, useRef } from 'react';
import { User, Mail, FileText, Save, Upload, AlertCircle, Smile } from 'lucide-react';
import { UserProfile, CharacterType } from '../types';
import { CharacterIllustration } from '../components/Illustrations';

interface ProfileProps {
  profile: UserProfile;
  setProfile: React.Dispatch<React.SetStateAction<UserProfile>>;
}

export const Profile: React.FC<ProfileProps> = ({ profile, setProfile }) => {
  const [error, setError] = useState<string>('');
  const fileInputRef = useRef<HTMLInputElement>(null);
  
  const validateEmail = (email: string) => {
    const re = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
    return re.test(email);
  };

  const handleSubmit = (e: React.FormEvent) => {
    e.preventDefault();
    if (!validateEmail(profile.email)) {
      setError('Please enter a valid email address.');
      return;
    }
    setError('');
    alert("Profile saved! The AI now knows who you are and what you look like.");
  };

  const handleFileUpload = (e: React.ChangeEvent<HTMLInputElement>) => {
    const file = e.target.files?.[0];
    if (file) {
      const reader = new FileReader();
      reader.onload = (event) => {
        const content = event.target?.result as string;
        setProfile(prev => ({
          ...prev,
          documentName: file.name,
          documentContent: content
        }));
      };
      reader.readAsText(file);
    }
  };

  return (
    <div className="p-8 max-w-3xl mx-auto">
      <h1 className="text-3xl font-bold text-white mb-8">Your Profile</h1>
      
      <div className="bg-card border border-gray-800 rounded-2xl p-8 shadow-xl">
        <form onSubmit={handleSubmit} className="space-y-6">
          
          <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
            <div>
              <label className="block text-sm font-medium text-gray-400 mb-2 flex items-center gap-2">
                <User className="w-4 h-4" /> Full Name
              </label>
              <input
                type="text"
                value={profile.name}
                onChange={(e) => setProfile({ ...profile, name: e.target.value })}
                className="w-full bg-dark border border-gray-700 rounded-xl px-4 py-3 text-white focus:ring-2 focus:ring-primary outline-none"
              />
            </div>
            
            <div>
              <label className="block text-sm font-medium text-gray-400 mb-2 flex items-center gap-2">
                <Mail className="w-4 h-4" /> Email
              </label>
              <input
                type="email"
                value={profile.email}
                onChange={(e) => {
                    setProfile({ ...profile, email: e.target.value });
                    if (error) setError('');
                }}
                className={`w-full bg-dark border rounded-xl px-4 py-3 text-white focus:ring-2 outline-none ${error ? 'border-red-500 focus:ring-red-500' : 'border-gray-700 focus:ring-primary'}`}
              />
              {error && <p className="text-red-500 text-xs mt-1 flex items-center gap-1"><AlertCircle className="w-3 h-3"/> {error}</p>}
            </div>
          </div>
          
           <div>
            <label className="block text-sm font-medium text-gray-400 mb-2 flex items-center gap-2">
                <Smile className="w-4 h-4" /> Choose Your Character
            </label>
            <div className="grid grid-cols-3 md:grid-cols-6 gap-4 p-4 bg-dark rounded-xl border border-gray-700">
                {Object.values(CharacterType).map((char) => (
                    <button
                        type="button"
                        key={char}
                        onClick={() => setProfile({ ...profile, character: char })}
                        className={`flex flex-col items-center justify-center p-2 rounded-lg transition-all ${profile.character === char ? 'bg-primary/20 ring-2 ring-primary' : 'hover:bg-gray-700'}`}
                    >
                        <CharacterIllustration character={char} mood="Idle" className="w-16 h-16" />
                        <span className={`mt-2 text-xs font-medium ${profile.character === char ? 'text-white' : 'text-gray-400'}`}>{char}</span>
                    </button>
                ))}
            </div>
           </div>

          <div>
            <label className="block text-sm font-medium text-gray-400 mb-2 flex items-center gap-2">
              <FileText className="w-4 h-4" /> Bio / Summary
            </label>
            <textarea
              value={profile.bio}
              onChange={(e) => setProfile({ ...profile, bio: e.target.value })}
              className="w-full bg-dark border border-gray-700 rounded-xl px-4 py-3 text-white h-32 resize-none focus:ring-2 focus:ring-primary outline-none leading-relaxed mb-4"
              placeholder="E.g., I am a software engineer looking for backend roles..."
            />
            
            <div className="bg-dark border border-gray-700 border-dashed rounded-xl p-6 text-center">
               <input 
                 type="file" 
                 ref={fileInputRef}
                 className="hidden" 
                 accept=".txt,.md,.csv,.json"
                 onChange={handleFileUpload}
               />
               <div className="flex flex-col items-center justify-center gap-2">
                  {profile.documentName ? (
                      <div className="flex items-center gap-2 text-green-400 bg-green-400/10 px-4 py-2 rounded-lg">
                          <FileText className="w-5 h-5" />
                          <span className="font-medium">{profile.documentName}</span>
                          <button 
                            type="button"
                            onClick={(e) => {
                                e.stopPropagation();
                                setProfile(p => ({ ...p, documentName: undefined, documentContent: undefined }));
                                if(fileInputRef.current) fileInputRef.current.value = '';
                            }}
                            className="ml-2 hover:text-white"
                          >
                              ×
                          </button>
                      </div>
                  ) : (
                    <>
                        <div className="p-3 bg-gray-800 rounded-full">
                            <Upload className="w-6 h-6 text-primary" />
                        </div>
                        <div className="text-sm text-gray-400">
                            <button 
                                type="button"
                                onClick={() => fileInputRef.current?.click()}
                                className="text-primary font-bold hover:underline"
                            >
                                Upload a document
                            </button> 
                            <span className="ml-1">to provide context</span>
                        </div>
                        <p className="text-xs text-gray-600">Supports .txt, .md (Text files)</p>
                    </>
                  )}
               </div>
            </div>
          </div>

          <div className="flex justify-end pt-4">
            <button
              type="submit"
              className="flex items-center gap-2 bg-primary hover:bg-indigo-600 text-white px-8 py-3 rounded-xl font-bold shadow-lg shadow-primary/25 transition-all"
            >
              <Save className="w-4 h-4" />
              Save Profile
            </button>
          </div>
        </form>
      </div>
    </div>
  );
};


Overwriting frontend/src/views/Profile.tsx


In [57]:
%%writefile frontend/src/views/Progress.tsx
import React, { useState } from 'react';
import { Clock, AlertCircle, CheckSquare, ChevronDown, ChevronUp, ArrowRight, Loader2, Trash2 } from 'lucide-react';
import { format, addHours, addDays } from 'date-fns';
import { Task, TaskStatus, AIAccessLevel, TaskCategory, UserProfile } from '../types';
import { CATEGORY_COLORS } from '../constants';
import { QuizModal } from '../components/AI/QuizModal';
import { HelpModal } from '../components/AI/HelpModal';
import { CharacterIllustration } from '../components/Illustrations';

interface ProgressProps {
  tasks: Task[];
  setTasks: React.Dispatch<React.SetStateAction<Task[]>>;
  userProfile: UserProfile;
}

export const Progress: React.FC<ProgressProps> = ({ tasks, setTasks, userProfile }) => {

  // ACTIVE TASKS
  const activeTasks = tasks.filter(
    t => t.status === TaskStatus.IN_PROGRESS || t.status === TaskStatus.FAILED_VERIFICATION
  );
  const completedTasks = tasks.filter(t => t.status === TaskStatus.COMPLETED);

  const [expandedTaskId, setExpandedTaskId] = useState<string | null>(null);
  const [checkInTask, setCheckInTask] = useState<Task | null>(null);

  // VERIFICATION LOADING
  const [isVerifying, setIsVerifying] = useState(false);

  // QUIZ
  const [quizLoading, setQuizLoading] = useState(false);
  const [quizQuestions, setQuizQuestions] = useState<any[]>([]);
  const [showQuiz, setShowQuiz] = useState(false);

  // HELP
  const [helpLoading, setHelpLoading] = useState(false);
  const [helpContent, setHelpContent] = useState<any>(null);
  const [showHelp, setShowHelp] = useState(false);

  const toggleExpand = (id: string) => {
    setExpandedTaskId(expandedTaskId === id ? null : id);
  };

  // Extend Deadline
  const handleExtendDeadline = (taskId: string, duration: { hours?: number; days?: number }) => {
    setTasks(prev =>
      prev.map(t => {
        if (t.id !== taskId) return t;
        const current = new Date(`${t.dueDate}T${t.dueTime}`);
        const newDate = duration.hours
          ? addHours(current, duration.hours)
          : addDays(current, duration.days || 0);

        return {
          ...t,
          dueDate: format(newDate, 'yyyy-MM-dd'),
          dueTime: format(newDate, 'HH:mm'),
        };
      })
    );
  };

  // Delete Task
  const handleDeleteTask = (taskId: string) => {
    if (window.confirm('Delete this task permanently?')) {
      setTasks(prev => prev.filter(t => t.id !== taskId));
    }
  };

  // BACKEND HELP CALL
  const launchHelp = async (task: Task) => {
    setHelpLoading(true);
    setShowHelp(true);

    const response = await fetch("http://127.0.0.1:8000/help", {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify({
        task: task
      })
    });

    const data = await response.json();
    setHelpContent(data.help || data);
    setHelpLoading(false);
  };

  const initiateCheckIn = (task: Task) => {
    setCheckInTask(task);
  };

  const handleVerificationFail = (task: Task) => {
    setTasks(prev =>
      prev.map(t =>
        t.id === task.id
          ? { ...t, status: TaskStatus.FAILED_VERIFICATION, verificationFailedCount: t.verificationFailedCount + 1 }
          : t
      )
    );
  };

  const completeTask = (taskId: string) => {
    setTasks(prev =>
      prev.map(t =>
        t.id === taskId
          ? { ...t, status: TaskStatus.COMPLETED, completedAt: new Date().toISOString() }
          : t
      )
    );
    setShowQuiz(false);
  };

  // POST /task → normalizes + returns quiz/help route
  const requestQuizFromBackend = async (task: Task, contextUrl?: string) => {
    const response = await fetch("http://127.0.0.1:8000/task", {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify({
        title: task.title,
        description: task.description,
        category: task.category,
      })
    });

    const data = await response.json();

    if (data.stage === "quiz") {
      setQuizQuestions(data.quiz.questions);
      setShowQuiz(true);
    } else if (data.stage === "help") {
      launchHelp(task);
    }
  };

  // POST /verify → networking verification
  const requestNetworkingVerification = async (names: string[], task: Task) => {
    const response = await fetch("http://127.0.0.1:8000/verify-networking", {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify({ names, task })
    });

    const data = await response.json();
    return data.verified;
  };

  // POST /submit-quiz
  const gradeQuiz = async (normalizedTask: any, quiz: any, userAnswers: number[]) => {
    const response = await fetch("http://127.0.0.1:8000/submit-quiz", {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify({
        normalized_task: normalizedTask,
        quiz,
        answers: userAnswers
      })
    });

    return await response.json();
  };

  // CHECK-IN RESPONSE
  const handleCheckInResponse = async (yes: boolean) => {
    if (!checkInTask) return;

    if (!yes) {
      const t = checkInTask;
      setCheckInTask(null);
      return launchHelp(t);
    }

    setIsVerifying(true);

    try {
      // Networking
      if (checkInTask.category === TaskCategory.NETWORKING) {
        const names = prompt("Enter recruiter names (comma-separated):");
        if (!names) return;

        const verified = await requestNetworkingVerification(names.split(','), checkInTask);

        if (verified) {
          completeTask(checkInTask.id);
        } else {
          handleVerificationFail(checkInTask);
          launchHelp(checkInTask);
        }
      }

      // Research / General Tasks
      else {
        await requestQuizFromBackend(checkInTask);
      }
    } catch (err) {
      console.error(err);
      alert("Verification failed.");
    }

    setIsVerifying(false);
    setCheckInTask(null);
  };

  // ---------------- RENDER ------------------

  return (
    <div className="p-8 max-w-4xl mx-auto pb-20">

      <h1 className="text-3xl font-bold text-white mb-8">Task Progress</h1>

      {/* ACTIVE TASKS */}
      <div className="space-y-6">
        {activeTasks.length === 0 && (
          <div className="text-center py-16 text-gray-500 flex flex-col items-center">
            <CharacterIllustration character={userProfile.character} mood="Success" className="w-40 h-40 mb-4" />
            <p className="font-medium text-lg text-gray-300">No active tasks!</p>
            <p className="text-sm">Take a break — you earned it.</p>
          </div>
        )}

        {activeTasks.map(task => (
          <div key={task.id} className={`bg-card border ${task.status === TaskStatus.FAILED_VERIFICATION ? 'border-red-500/50' : 'border-gray-800'} rounded-2xl`}>

            {/* TOP BLOCK */}
            <div className="p-6 flex flex-col md:flex-row md:items-center justify-between gap-4 cursor-pointer"
                 onClick={() => toggleExpand(task.id)}>
              <div className="flex items-start gap-4">
                <div className={`mt-1 w-3 h-3 rounded-full ${CATEGORY_COLORS[task.category]}`} />
                <div>
                  <h3 className="text-xl font-bold text-white">{task.title}</h3>
                  <p className="text-sm text-gray-400 mt-1 flex items-center gap-2">
                    <Clock className="w-4 h-4" /> Due: {task.dueDate} at {task.dueTime}
                  </p>
                </div>
              </div>

              <div className="flex items-center gap-3">
                <button
                  onClick={(e) => { e.stopPropagation(); initiateCheckIn(task); }}
                  className="bg-primary hover:bg-indigo-600 text-white px-5 py-2 rounded-lg text-sm font-bold">
                  Check In
                </button>
                {expandedTaskId === task.id ? <ChevronUp /> : <ChevronDown />}
              </div>
            </div>

            {/* EXPANDED BLOCK */}
            {expandedTaskId === task.id && (
              <div className="bg-gray-900/50 border-t border-gray-800 p-6 space-y-6">

                <div>
                  <h4 className="text-sm font-bold text-gray-400 mb-2">Description</h4>
                  <p className="text-gray-300">{task.description}</p>
                </div>

                {/* DEADLINE & ACTIONS */}
                <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
                  <div>
                    <h4 className="text-sm font-bold text-gray-400 mb-2">Extend Deadline</h4>
                    <div className="flex gap-2">
                      <button onClick={() => handleExtendDeadline(task.id, { hours: 1 })} className="px-3 py-1 bg-gray-800">+1 Hr</button>
                      <button onClick={() => handleExtendDeadline(task.id, { hours: 3 })} className="px-3 py-1 bg-gray-800">+3 Hrs</button>
                      <button onClick={() => handleExtendDeadline(task.id, { days: 1 })} className="px-3 py-1 bg-gray-800">+1 Day</button>
                    </div>
                  </div>

                  <div>
                    <h4 className="text-sm font-bold text-gray-400 mb-2">Actions</h4>
                    <button onClick={() => handleDeleteTask(task.id)} className="flex items-center gap-2 px-3 py-1 bg-red-900/50 text-red-400 rounded">
                      <Trash2 className="w-3 h-3" /> Delete Task
                    </button>
                  </div>
                </div>

                {/* HELP */}
                <div className="flex items-center justify-between bg-gray-800/30 p-4 rounded-xl">
                  <div className="flex items-center gap-3">
                    <AlertCircle className="text-secondary w-5 h-5" />
                    <span className="text-sm text-gray-300">Need help? Let's do it together.</span>
                  </div>
                  <button
                    onClick={() => launchHelp(task)}
                    className="text-secondary font-bold text-sm flex items-center gap-1">
                    Let's do it <ArrowRight className="w-4 h-4" />
                  </button>
                </div>

              </div>
            )}

          </div>
        ))}
      </div>

      {/* COMPLETED TASKS */}
      {completedTasks.length > 0 && (
        <div className="mt-12">
          <h2 className="text-xl font-bold text-gray-400 mb-4">Completed</h2>
          {completedTasks.map(task => (
            <div key={task.id} className="bg-gray-900 border border-gray-800 p-4 rounded-xl flex items-center gap-4 opacity-60">
              <CheckSquare className="text-green-500 w-6 h-6" />
              <span className="text-gray-400 line-through text-lg">{task.title}</span>
              <span className="ml-auto text-xs text-gray-600">
                {task.completedAt && new Date(task.completedAt).toLocaleDateString()}
              </span>
            </div>
          ))}
        </div>
      )}

      {/* CHECK-IN MODAL */}
      {checkInTask && (
        <div className="fixed inset-0 bg-black/80 flex items-center justify-center z-50 p-4">
          <div className="bg-card border border-gray-700 rounded-2xl p-8 max-w-md w-full text-center">
            {isVerifying ? (
              <>
                <Loader2 className="w-16 h-16 text-primary animate-spin mx-auto mb-4" />
                <h3 className="text-xl font-bold text-white">Verifying...</h3>
                <p className="text-gray-400">Checking your work...</p>
              </>
            ) : (
              <>
                <h2 className="text-2xl font-bold text-white mb-4">Reality Check</h2>
                <p className="text-gray-300 mb-8 text-lg">
                  Have you completed <span className="text-primary">"{checkInTask.title}"</span>?
                </p>
                <div className="grid grid-cols-2 gap-4">
                  <button
                    onClick={() => handleCheckInResponse(false)}
                    className="py-3 rounded-xl border border-gray-600 text-gray-300 hover:bg-gray-800">
                    No, help me
                  </button>
                  <button
                    onClick={() => handleCheckInResponse(true)}
                    className="py-3 rounded-xl bg-primary hover:bg-indigo-600 text-white font-bold">
                    Yes
                  </button>
                </div>
              </>
            )}
          </div>
        </div>
      )}

      {/* QUIZ MODAL */}
      <QuizModal
        isOpen={showQuiz}
        loading={quizLoading}
        questions={quizQuestions}
        character={userProfile.character}
        onComplete={(passed) => {
          const t = activeTasks.find(t => t.id === expandedTaskId) || activeTasks[0];
          if (t) {
            if (passed) completeTask(t.id);
            else handleVerificationFail(t);
          }
          setShowQuiz(false);
        }}
        onRequestHelp={() => {
          const t = activeTasks.find(t => t.id === expandedTaskId) || activeTasks[0];
          if (t) launchHelp(t);
          setShowQuiz(false);
        }}
      />

      {/* HELP MODAL */}
      <HelpModal
        isOpen={showHelp}
        loading={helpLoading}
        content={helpContent}
        character={userProfile.character}
        category={activeTasks.find(t => t.id === expandedTaskId)?.category || TaskCategory.OTHER}
        onClose={() => setShowHelp(false)}
        onGenerateMore={() => {
          const task = activeTasks.find(t => t.id === expandedTaskId);
          if (task) launchHelp(task);
        }}
        onFinishedLearning={() => {
          setShowHelp(false);
        }}
      />

    </div>
  );
};


Overwriting frontend/src/views/Progress.tsx


In [58]:
%%writefile frontend/src/App.tsx
import React, { useState, useEffect } from 'react';
import { Sidebar } from './components/Sidebar';
import { Home } from './views/Home';
import { Progress } from './views/Progress';
import { Profile } from './views/Profile';
import { Calendar } from './views/Calendar';
import { Documents } from './views/Documents';
import { Task, UserProfile, CharacterType, GeneratedDocument } from './types';
import { MOCK_INITIAL_TASKS } from './constants';

const API = "http://127.0.0.1:8000";

const App: React.FC = () => {
  const [activeTab, setActiveTab] = useState('home');
  const [tasks, setTasks] = useState<Task[]>(MOCK_INITIAL_TASKS);
  const [profile, setProfile] = useState<UserProfile>({
    name: 'Alex',
    email: 'alex@example.com',
    bio: 'I am a frontend developer learning AI integration and trying to network with tech leads in San Francisco.',
    character: CharacterType.MALE,
  });
  const [documents, setDocuments] = useState<GeneratedDocument[]>([]);

  // -------------------------------
  // LOAD DATA ON STARTUP
  // -------------------------------
  useEffect(() => {
    const loadData = async () => {
      try {
        const tRes = await fetch(`${API}/tasks`);
        const pRes = await fetch(`${API}/profile`);

        if (tRes.ok) setTasks(await tRes.json());
        if (pRes.ok) setProfile(await pRes.json());
      } catch (err) {
        console.error("Backend unavailable, using local defaults.");
      }
    };

    loadData();
  }, []);

  // -------------------------------
  // AUTO-SAVE TASKS
  // -------------------------------
  useEffect(() => {
    fetch(`${API}/save-tasks`, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(tasks),
    }).catch(() => {});
  }, [tasks]);

  // -------------------------------
  // AUTO-SAVE PROFILE
  // -------------------------------
  useEffect(() => {
    fetch(`${API}/save-profile`, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(profile),
    }).catch(() => {});
  }, [profile]);

  const renderContent = () => {
    switch (activeTab) {
      case 'home':
        return <Home tasks={tasks} setTasks={setTasks} userProfile={profile} setActiveTab={setActiveTab} />;
      case 'progress':
        return <Progress tasks={tasks} setTasks={setTasks} userProfile={profile} />;
      case 'calendar':
        return <Calendar tasks={tasks} setActiveTab={setActiveTab} />;
      case 'documents':
        return <Documents documents={documents} setDocuments={setDocuments} tasks={tasks} userProfile={profile} />;
      case 'profile':
        return <Profile profile={profile} setProfile={setProfile} />;
      default:
        return <Home tasks={tasks} setTasks={setTasks} userProfile={profile} setActiveTab={setActiveTab} />;
    }
  };

  return (
    <div className="flex min-h-screen bg-dark text-gray-100 font-sans selection:bg-primary selection:text-white">
      <Sidebar activeTab={activeTab} setActiveTab={setActiveTab} />
      <main className="flex-1 overflow-y-auto h-screen relative">
        {/* Background ambient glow */}
        <div className="fixed top-0 left-0 w-full h-full pointer-events-none overflow-hidden z-0">
           <div className="absolute top-[-10%] right-[-10%] w-[500px] h-[500px] bg-primary/10 rounded-full blur-[120px]"></div>
           <div className="absolute bottom-[-10%] left-[-10%] w-[500px] h-[500px] bg-secondary/10 rounded-full blur-[120px]"></div>
        </div>
        
        <div className="relative z-10">
          {renderContent()}
        </div>
      </main>
    </div>
  );
};

export default App;


Overwriting frontend/src/App.tsx


In [59]:
!mkdir -p backend/data


In [62]:
!pkill -f uvicorn
!uvicorn backend.main:app --host 0.0.0.0 --port 8000 --reload


INFO:     Will watch for changes in these directories: ['/kaggle/working']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [542] using StatReload
INFO:     Started server process [544]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [544]
INFO:     Stopping reloader process [542]


In [65]:
!pkill -f uvicorn


In [67]:
import uvicorn
import threading

def run_server():
    uvicorn.run(
        "backend.main:app",
        host="0.0.0.0",
        port=8000,
        reload=False,   # reload MUST be off in Kaggle
        log_level="info"
    )

# Kill old servers
!pkill -f uvicorn || echo "No old uvicorn servers."

# Start server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print("FastAPI server started in background thread on port 8000!")


FastAPI server started in background thread on port 8000!


INFO:     Started server process [431]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [68]:
import requests
requests.get("http://127.0.0.1:8000").json()


INFO:     127.0.0.1:35316 - "GET / HTTP/1.1" 200 OK


{'status': 'backend running'}

In [69]:
import requests

resp = requests.post(
    "http://127.0.0.1:8000/process-task",
    json={
        "title": "Learn React",
        "description": "I must study React fundamentals",
        "category": "RESEARCH"
    }
)

resp.json()



=== ORCHESTRATOR STEP: TaskAgent ===

=== RAW TASKAGENT OUTPUT ===
 {
  "normalizedTitle": "Learn React Fundamentals",
  "normalizedDescription": "Study React fundamentals to gain a solid understanding.",
  "category": "EDUCATION",
  "keywords": ["React", "fundamentals", "JavaScript", "front-end", "learn"],
  "complexity": 5,
  "nextAgent": "HelpAgent"
}
Normalized: {'normalizedTitle': 'Learn React Fundamentals', 'normalizedDescription': 'Study React fundamentals to gain a solid understanding.', 'category': 'EDUCATION', 'keywords': ['React', 'fundamentals', 'JavaScript', 'front-end', 'learn'], 'complexity': 5, 'nextAgent': 'HelpAgent'}

=== ORCHESTRATOR STEP: HelpAgent ===

=== RAW HELP MODEL OUTPUT ===
 {
  "summary": "Learn the fundamental concepts of React to build interactive user interfaces.",
  "steps": [
    "Set up your development environment with Node.js and npm or yarn.",
    "Learn about JSX and how it's used in React components.",
    "Understand components, props, and st

{'stage': 'help',
 'normalized': {'normalizedTitle': 'Learn React Fundamentals',
  'normalizedDescription': 'Study React fundamentals to gain a solid understanding.',
  'category': 'EDUCATION',
  'keywords': ['React', 'fundamentals', 'JavaScript', 'front-end', 'learn'],
  'complexity': 5,
  'nextAgent': 'HelpAgent'},
 'help': {'summary': 'Learn the fundamental concepts of React to build interactive user interfaces.',
  'steps': ['Set up your development environment with Node.js and npm or yarn.',
   "Learn about JSX and how it's used in React components.",
   'Understand components, props, and state.',
   'Practice handling events in React.',
   'Explore conditional rendering and lists.',
   'Familiarize yourself with React Hooks (useState, useEffect, etc.).',
   'Build small projects to reinforce your knowledge.'],
  'resources': [{'title': 'React Official Documentation',
    'url': 'https://react.dev/learn'},
   {'title': 'Scrimba React Course',
    'url': 'https://scrimba.com/learn/

In [ ]:
requests.post(
    "http://127.0.0.1:8000/generate-spreadsheet",
    json={
        "tasks": [
            {"title":"Test", "category":"OTHER", "status":"TODO", "dueDate":"2025-01-01"}
        ],
        "user": {"name":"Alex"}
    }
).json()


In [ ]:
requests.post(
    "http://127.0.0.1:8000/verify-networking",
    json={
        "names": ["John Doe"],
        "task": {"title": "Contact recruiters"}
    }
).json()


In [70]:
!npm install --prefix frontend
!npm run build --prefix frontend


⠙⠹npm error code ENOENT
⠹npm error syscall open
⠹npm error path /kaggle/working/frontend/package.json
⠹npm error errno -2
⠹npm error enoent Could not read package.json: Error: ENOENT: no such file or directory, open '/kaggle/working/frontend/package.json'
⠹npm error enoent This is related to npm not being able to find a file.
npm error enoent
⠹npm error A complete log of this run can be found in: /root/.npm/_logs/2025-12-01T11_15_03_756Z-debug-0.log
⠹npm error code ENOENT
npm error syscall open
npm error path /kaggle/working/frontend/package.json
npm error errno -2
npm error enoent Could not read package.json: Error: ENOENT: no such file or directory, open '/kaggle/working/frontend/package.json'
npm error enoent This is related to npm not being able to find a file.
npm error enoent
npm error A complete log of this run can be found in: /root/.npm/_logs/2025-12-01T11_15_05_378Z-debug-0.log
